## BRONZE TO SILVER LAYER


In [1]:
import pandas as pd
from common_utilities import replace_punctuation_from_columns, global_path

### Bronze Layer - Trade History


In [2]:
# Initialize an empty list to store DataFrames
df_TradeHistory = []

# Loop through List of all CSV files in the folder
for file_path in global_path.tradehistory_bronze_layer_path.glob("*.csv"):
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Append the DataFrame to the list
    df_TradeHistory.append(df)

# Concatenate all DataFrames into one
df_TradeHistory = pd.concat(df_TradeHistory, ignore_index=True)
df_TradeHistory.columns = replace_punctuation_from_columns(df_TradeHistory.columns)
df_TradeHistory.dropna(how="all", axis=1, inplace=True)

# Convert 'trade_num' to int
df_TradeHistory["trade_num"] = df_TradeHistory["trade_num"].fillna(0).astype(int)

# Remove the currency symbol and commas, then convert to float
df_TradeHistory["amount"] = (
    df_TradeHistory["amount"].replace("[₹,]", "", regex=True).astype(float)
)

df_TradeHistory["price"] = (
    df_TradeHistory["price"].replace("[₹,]", "", regex=True).astype(float)
)

# Add Datetime Col
df_TradeHistory["datetime"] = pd.to_datetime(
    df_TradeHistory["date"] + " " + df_TradeHistory["trade_time"].fillna("00:00:00"),
    format="%d-%m-%Y %H:%M:%S",
)

# Convert 'expiry' to desired string format
df_TradeHistory["expiry"] = pd.to_datetime(df_TradeHistory["expiry"], format="%d-%m-%Y")
df_TradeHistory["expiry"] = df_TradeHistory["expiry"].dt.strftime("%d%b%Y")

# Convert the 'side' column in df_TradeHistory to uppercase
df_TradeHistory["side"] = df_TradeHistory["side"].str.upper()


# Function to apply the conditional concatenation
def concat_stock_name(row):
    if row["instrument_type"] == "European Call":
        return (
            str(row["company"])
            + "-CE-"
            + str(row["strike_price"])
            + "-"
            + row["expiry"]
        )
    elif row["instrument_type"] == "European Put":
        return (
            str(row["company"])
            + "-PE-"
            + str(row["strike_price"])
            + "-"
            + row["expiry"]
        )
    else:
        return str(row["company"])


# Apply the function to the DataFrame
df_TradeHistory["stock_name"] = df_TradeHistory.apply(concat_stock_name, axis=1)

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["datetime", "stock_name"])


df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "exchange",
        "segment",
        "stock_name",
        "scrip_code",
        "side",
        "amount",
        "quantity",
        "price",
    ]
]

# Save the result as a csv file
df_TradeHistory.to_csv(global_path.tradehistory_silver_file_path, index=None)
df_TradeHistory.info()

print("SILVER Layer csv file for trade history successfully created at:")
print(global_path.tradehistory_silver_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 17 to 43
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    171 non-null    datetime64[ns]
 1   exchange    171 non-null    object        
 2   segment     171 non-null    object        
 3   stock_name  171 non-null    object        
 4   scrip_code  171 non-null    object        
 5   side        171 non-null    object        
 6   amount      171 non-null    float64       
 7   quantity    171 non-null    float64       
 8   price       171 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 13.4+ KB
SILVER Layer csv file for trade history successfully created at:
/storage/emulated/0/Upstox/DATA/SILVER/TradeHistory/TradeHistory_data.csv


/data/data/com.termux/files/usr/tmp/ipykernel_26519/2607471260.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_TradeHistory = pd.concat(df_TradeHistory, ignore_index=True)
